<a href="https://colab.research.google.com/github/gdbwoo/wrangling/blob/main/assignment/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: Data Wrangling
## `! git clone https://github.com/DS3001/wrangling`

In [1]:
! git clone https://github.com/gdbwoo/wrangling

Cloning into 'wrangling'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 94 (delta 40), reused 13 (delta 10), pack-reused 40
Receiving objects: 100% (94/94), 6.27 MiB | 24.49 MiB/s, done.
Resolving deltas: 100% (46/46), done.


**Q1.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q1.**

**Part 1:** Wickham explains in the abstract that his research for this particular paper focuses on data tidying, specifically "how to make data cleaning as easy and effective as possible" so that datasets are easier to work with, and can help in discovering further findings and insights. He proposes a specific structure: "each variable is a column, each observation is a row, and each type of observational unit is a table". This framework allows data scientists to deal with a wide range of untidy datasets with only a small set of tools and also helps in tidy tool development in data analysis. A consistent data structure and tools provide many advantageous as shown by the paper.

**Part 2:** The tidy data standard and its principles are intended to provide a standard way to organize a dataset which makes it easier to initialize data cleaning instead of having to start from scratch each time. According to Wickham's introduction, "the tidy data standard \[was\] designed to facilitate intial exploration and analysis of the data, and to simplify the development of data analysis tools that work well together".

**Part 3:** The sentence, "Like families, tidy datasets are all alike but every messy dataset is messy in its own way," seems to say how all tidy datasets are cleaned/organized so they have similar properties and traits since they have been through human manipulation, whereas messy datasets are all unique, having their own different problems, discoveries, insights, etc. The second sentence, "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general," seems to say how the person working with the data and their previous knowledge/assumptions makes a big difference in how the data is handled and understood because they determine certain relationships among and between the data.

**Part 4:** Wickham defines values as numbers if quantitative, and as strings if qualitative, and a collection of these values make up a dataset. Wickham also defines variables and observations as the two ways that values are organized, where every value is both a variable and an observation. A variable "contains all values that measure the same underlying attribute", whereas an observation "contains all values measured on the same unit across attributes".

**Part 5:** In section 2.3, tidy data is defined as "a standard way of mapping the meaning of a dataset to its structure" and has the following properties:
1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

**Part 6:** According to the intro to Section 3 and Section 3.1, the 5 most common problems with messy datasets are the following:
1. Column headers are values, not variable names
2. Multiple variables are stored in one column
3. Variables are stored in both rows and columns
4. Multiple types of observational units are stored in the same table
5. A single observational unit is stored in multiple tables

Table 4 is messy because the column headers are values instead of variable names. To fix this, income variable should have a separate column of its own and instead of splitting the income as column headers to show the count, there should be another seperate column for the count/frequency.

"Melting" or "stacking" a dataset means to convert columns into rows.

**Part 7:** Like Table 4, Table 11 is messy because the column headers are values instead of variable names, whereas Table 12 has a single variable, "date" for the day. However, Table 12a still needs cleaning since the "element" column contains names of variables, "tmax" and "tmin". In order to fix this, Table 12b splits the "element" column into two separate columns: "tmax" and "tmin".

**Part 8:** The "chicken-and-egg" problem with focusing on tidy data is that "if tidy data is only as useful as the tools that work with it, then tidy tools will be inextricably linked to tidy data". In other words, there is less flexibility in choosing/changing tools. Wickham hopes that more focus would be spent on studying data cleaning and its process, so that more types of frameworks could be developed and the task itself would become easier through these emerging tools.

**Q2.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the `./data/sharks.csv` data covered in the lecture, clean the "Type" variable as well as you can, and explain the choices you make.
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

In [3]:
# Q2.
# Referenced the professor's provided file on github: https://github.com/DS3001/wrangling/blob/main/assignment/assignment_solutions.ipynb for help and ideas

# Q2 Part 1
adf = pd.read_csv('wrangling/assignment/data/airbnb_hw.csv', low_memory=False)
print(adf.shape, '\n')
adf.head()

(30478, 13) 



,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,5162530,NaN,1 Bedroom in Prime Williamsburg,Brooklyn,Apartment,NaN,Entire home/apt,11249.0,1.0,1,0,145,NaN
1,33134899,NaN,"Sunny, Private room in Bushwick",Brooklyn,Apartment,NaN,Private room,11206.0,1.0,1,1,37,NaN
2,39608626,NaN,Sunny Room in Harlem,Manhattan,Apartment,NaN,Private room,10032.0,1.0,1,1,28,NaN
3,500,6/26/2008,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
4,500,6/26/2008,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0


In [4]:
# Making another copy of just the Price column and taking a look at that data
price = adf['Price']
print(price.unique(), '\n')

['145' '37' '28' '199' '549' '149' '250' '90' '270' '290' '170' '59' '49'
 '68' '285' '75' '100' '150' '700' '125' '175' '40' '89' '95' '99' '499'
 '120' '79' '110' '180' '143' '230' '350' '135' '85' '60' '70' '55' '44'
 '200' '165' '115' '74' '84' '129' '50' '185' '80' '190' '140' '45' '65'
 '225' '600' '109' '1,990' '73' '240' '72' '105' '155' '160' '42' '132'
 '117' '295' '280' '159' '107' '69' '239' '220' '399' '130' '375' '585'
 '275' '139' '260' '35' '133' '300' '289' '179' '98' '195' '29' '27' '39'
 '249' '192' '142' '169' '1,000' '131' '138' '113' '122' '329' '101' '475'
 '238' '272' '308' '126' '235' '315' '248' '128' '56' '207' '450' '215'
 '210' '385' '445' '136' '247' '118' '77' '76' '92' '198' '205' '299'
 '222' '245' '104' '153' '349' '114' '320' '292' '226' '420' '500' '325'
 '307' '78' '265' '108' '123' '189' '32' '58' '86' '219' '800' '335' '63'
 '229' '425' '67' '87' '1,200' '158' '650' '234' '310' '695' '400' '166'
 '119' '62' '168' '340' '479' '43' '395' '144' '52' 

In [5]:
# The prices are in strings and the ones that are >= 1000 have a comma to indicate thousands, etc.

# Get rid of all commas by replacing with nothing so we can convert the strings into numbers
price = price.str.replace(',','')

# Convert strings to numbers for future calculations/operations we want to do on price and set any invalid parsing as NaN
price = pd.to_numeric(price,errors='coerce')

# New price column after cleaning
print(price.unique(), '\n')

# Listing total missing values
print('Total missing: ', sum( price.isnull()))

# As we can see by the results below, there are 0 missing values.

[  145    37    28   199   549   149   250    90   270   290   170    59
    49    68   285    75   100   150   700   125   175    40    89    95
    99   499   120    79   110   180   143   230   350   135    85    60
    70    55    44   200   165   115    74    84   129    50   185    80
   190   140    45    65   225   600   109  1990    73   240    72   105
   155   160    42   132   117   295   280   159   107    69   239   220
   399   130   375   585   275   139   260    35   133   300   289   179
    98   195    29    27    39   249   192   142   169  1000   131   138
   113   122   329   101   475   238   272   308   126   235   315   248
   128    56   207   450   215   210   385   445   136   247   118    77
    76    92   198   205   299   222   245   104   153   349   114   320
   292   226   420   500   325   307    78   265   108   123   189    32
    58    86   219   800   335    63   229   425    67    87  1200   158
   650   234   310   695   400   166   119    62   

In [6]:
# Q2 Part 2

sdf = pd.read_csv('wrangling/assignment/data/sharks.csv', low_memory=False)
print(sdf.shape, '\n')
sdf.head()

(6462, 257) 



,index,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255
0,0,2020.02.05,05-Feb-2020,2020.0,Unprovoked,USA,Maui,NaN,Stand-Up Paddle boarding,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2020.01.30.R,Reported 30-Jan-2020,2020.0,Provoked,BAHAMAS,Exumas,NaN,Floating,Ana Bruna Avila,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020.01.17,17-Jan-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,Windang Beach,Surfing,Will Schroeter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2020.01.16,16-Jan-2020,2020.0,Unprovoked,NEW ZEALAND,Southland,Oreti Beach,Surfing,Jordan King,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2020.01.13,13-Jan-2020,2020.0,Unprovoked,USA,North Carolina,"Rodanthe, Dare County",Surfing,Samuel Horne,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Making another copy of just the Type column and taking a look at that data
attackType = sdf['Type']
print(attackType.unique(), '\n')

['Unprovoked' 'Provoked' 'Questionable' 'Watercraft' 'Unconfirmed'
 'Unverified' 'Invalid' 'Under investigation' 'Boating' 'Sea Disaster' nan
 'Boat' 'Boatomg'] 



In [8]:
# Seeing the occurences of each attackType in this dataset
attackType.value_counts()

Unprovoked             4716
Provoked                593
Invalid                 552
Sea Disaster            239
Watercraft              142
Boat                    109
Boating                  92
Questionable             10
Unconfirmed               1
Unverified                1
Under investigation       1
Boatomg                   1
Name: Type, dtype: int64

In [9]:
# Grouping/combining attacks into broader categories

# Watercraft
attackType = attackType.replace(['Sea Disaster', 'Boat','Boating','Boatomg'],'Watercraft')
attackType.value_counts()

# Under Investigation
attackType = attackType.replace(['Invalid', 'Questionable','Unconfirmed','Unverified','Under investigation'],np.nan)
attackType.value_counts()

sdf['Type'] = attackType # Replace the 'Type' variable with the cleaned version, attackType
del attackType # Destroy the temporary vector, attackType
sdf['Type'].value_counts()

Unprovoked    4716
Provoked       593
Watercraft     583
Name: Type, dtype: int64

In [10]:
# Referenced the professor's provided file on github: https://github.com/DS3001/wrangling/blob/main/assignment/assignment_solutions.ipynb for the below piece of code

sdf['Fatal (Y/N)'] = sdf['Fatal (Y/N)'].replace(['UNKNOWN', 'F','M','2017'],np.nan) # All unclean values
sdf['Fatal (Y/N)'] = sdf['Fatal (Y/N)'].replace('y','Y') # All unclean values
pd.crosstab(sdf['Type'],sdf['Fatal (Y/N)'],normalize='index')

# As shown by the results below, we can see that shark attacks are much more likely to be fatal if the attack was unprovoked

Fatal (Y/N),N,Y
Type,,
Provoked,0.967521,0.032479
Unprovoked,0.743871,0.256129
Watercraft,0.684303,0.315697


In [11]:
# Q2 Part 3

url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
df = pd.read_csv(url,low_memory=False) # Pandas downloads and loads the .csv file for you
print(df.shape, '\n')
df.head()

(22986, 709) 



,InternalStudyID,REQ_REC#,Defendant_Sex,Defendant_Race,Defendant_BirthYear,Defendant_Age,Defendant_AgeGroup,Defendant_AgeatCurrentArrest,Defendant_AttorneyTypeAtCaseClosure,Defendant_IndigencyStatus,...,NewFelonySexualAssaultArrest_OffDate,NewFelonySexualAssaultArrest_ArrestDate,NewFelonySexualAssaultArrest_DaysBetweenContactEventandOffDate,NewFelonySexualAssaultArrest_DaysBetweenOffDateandArrestDate,NewFelonySexualAssaultArrest_DaysBetweenReleaseDateandOffDate,NewFelonySexualAssaultArrest_Disposition,Intertnalindicator_ReasonforExcludingFromFollowUpAnalysis,CriminalHistoryRecordsReturnedorCMSRecordsFoundforIndividual,DispRecordFoundforChargesinOct2017Contact_Atleast1dispfound,CrimeCommission2021ReportClassificationofDefendants
0,ADI00001,1,M,W,1986,31,3,31,99,99,...,,,,999,999,,4,1,0,Defendant could not be classified or tracked d...
1,ADI00007,3,M,B,1956,60,6,60,9,9,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
2,ADI00008,4,M,W,1990,27,3,27,9,9,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
3,CDI00036,6,M,B,1989,27,3,27,0,0,...,,,,999,999,,5,1,1,Defendant Detained Entire Pre-Trial Period_Und...
4,CDI00038,7,F,W,1988,28,3,28,0,0,...,,,,999,999,,0,1,1,New criminal offense punishable by incarcerati...


In [14]:
# Making another copy of just the WhetherDefendantWasReleasedPretrial column and taking a look at that data
release = df['WhetherDefendantWasReleasedPretrial']
print(release.unique(), '\n')

[9 0 1] 



In [15]:
# Seeing the value counts of WhetherDefendantWasReleasedPretrial in this dataset
print(release.value_counts(),'\n')

1    19154
0     3801
9       31
Name: WhetherDefendantWasReleasedPretrial, dtype: int64 



In [16]:
# The codebook showed a table:
# Label: Code
# 15-17 years of age: 1
# 18-25 years of age: 2
# 26-35 years of age: 3
# 36-45 years of age: 4
# 46-55 years of age: 5
# 56-65 years of age: 6
# Over 65 years of age: 7
# Unknown/missing: 9

# Since the unknown/missing table is code 9, replace 9 with np.nan
release = release.replace(9,np.nan) # In the codebook, the 9's are "unclear"
print(release.value_counts(),'\n')

# Replace original column with cleaned column
df['WhetherDefendantWasReleasedPretrial'] = release

1.0    19154
0.0     3801
Name: WhetherDefendantWasReleasedPretrial, dtype: int64 



In [21]:
# Q2 Part 4

# Making another copy of the ImposedSentenceAllChargeInContactEvent and SentenceTypeAllChargesAtConvictionInContactEvent column and taking a look at those data
sentence = df['ImposedSentenceAllChargeInContactEvent']
stype = df['SentenceTypeAllChargesAtConvictionInContactEvent']

print(sentence.unique(), '\n') # Some values are ' ', denoting missing

[' ' '60' '12' '.985626283367556' '36' '6' '24' '5.91375770020534' '120'
 '72' '11.9917864476386' '0' '2.95687885010267' '84' '108' '300' '240'
 '180' '4' '96' '2' '54' '.328542094455852' '44' '5' '115' '132' '48'
 '258' '34' '76' '.164271047227926' '.131416837782341' '111' '9' '3'
 '1.97125256673511' '36.9856262833676' '.0657084188911704'
 '35.4928131416838' '106.492813141684' '8' '35' '18.3141683778234' '480'
 '32' '93' '234' '732' '1.16427104722793' '4.6570841889117' '21' '7'
 '4.49281314168378' '18' '600' '43.1642710472279' '179' '52' '30' '20'
 '192' '702' '14' '55' '53' '11.9055441478439' '114' '35.0061601642711'
 '68' '.657084188911704' '46.6242299794661' '102' '65' '200' '57'
 '24.3285420944559' '12.1642710472279' '117' '81.4928131416838'
 '22.4928131416838' '1980' '3.6570841889117' '56' '10' '2.79260780287474'
 '1' '47' '22' '1500' '40' '284' '11' '118' '42' '162' '156'
 '47.2956878850103' '105' '51' '246' '29' '75' '324' '360'
 '34.4804928131417' '120.328542094456' '59.926078

In [22]:
# Convert strings to numbers for future calculations/operations we want to do and set any invalid parsing as nan
sentence = pd.to_numeric(sentence, errors='coerce')
print(np.sum(sentence.isnull()),'\n') # 9053 missing values of 23k, not so good

9053 



In [24]:
# Referenced the professor's provided file on github: https://github.com/DS3001/wrangling/blob/main/assignment/assignment_solutions.ipynb

print(pd.crosstab(sentence.isnull(), stype), '\n') # Category 4 is cases where the charges were dismissed

sentence = sentence.mask( stype == 4, 0) # Replace length with 0 when type ==4
sentence = sentence.mask( stype == 9, np.nan) # Replace length with np.nan when type == 9

print(pd.crosstab(sentence.isnull(), stype), '\n')
print(np.sum(sentence.isnull()),'\n') # now 274 missing instead of 9053

# Replace original column with cleaned column
df['ImposedSentenceAllChargeInContactEvent'] = sentence

SentenceTypeAllChargesAtConvictionInContactEvent     0     1    2     4    9
ImposedSentenceAllChargeInContactEvent                                      
False                                             8720  4299  914     0    0
True                                                 0     0    0  8779  274 

SentenceTypeAllChargesAtConvictionInContactEvent     0     1    2     4    9
ImposedSentenceAllChargeInContactEvent                                      
False                                             8720  4299  914  8779    0
True                                                 0     0    0     0  274 

274 



**Q3.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?